# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /Users/andyyu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/andyyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andyyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('labeled_messages', engine)
X = df['message']
y = df[list(df.columns[4:])]

In [4]:
category_names = y.columns.tolist()

In [5]:
len(category_names)

35

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):

    """
    Tokenize function which will be used in CountVectorizer;
    Steps:
    (1) lower the case
    (2) replace url
    (3) remove punctuations
    (4) tokenize
    (5) remove stopwords
    (6) reduce words to their root form
    Output is a list of cleaned tokens
    """
    text = text.lower() #lower case
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
    
    tokens = word_tokenize(text) #tokenize
    lemmatizer = WordNetLemmatizer() 
    
    #remove stopwords
    clean_tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    #reduce words to their root form
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in clean_tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [8]:
clf = MultinomialNB()

In [9]:
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('clf', MultiOutputClassifier(clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1f32fb00>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0,
                                          

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
from sklearn.metrics import classification_report, make_scorer, precision_recall_fscore_support, accuracy_score, f1_score

In [14]:
def evaluate_model(y_test, y_pred, category_names = category_names):
    metrics = pd.DataFrame(columns=['Category', 'f_score', 'accuracy','precision', 'recall'])

    for idx in range(len(category_names)):
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[category_names[idx]], y_pred[:,idx], average='weighted')
        accuracy = accuracy_score(y_test[category_names[idx]], y_pred[:,idx])

        metrics.at[idx, 'Category'] = category_names[idx]
        metrics.at[idx, 'f_score'] = f_score
        metrics.at[idx, 'accuracy'] = accuracy
        metrics.at[idx, 'precision'] = precision
        metrics.at[idx, 'recall'] = recall

    print('Average f_score:', metrics['f_score'].mean())
    print('Average accuracy:', metrics['accuracy'].mean())
    print('Average precision:', metrics['precision'].mean())
    print('Average recall:', metrics['recall'].mean())
    return metrics

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
results = evaluate_model(y_test, y_pred)
results

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average f_score: 0.8956232897394371
Average accuracy: 0.926325799136859
Average precision: 0.8862050130157928
Average recall: 0.926325799136859


,Category,f_score,accuracy,precision,recall
0,related,0.651694,0.756016,0.693701,0.756016
1,request,0.761565,0.834997,0.722718,0.834997
2,offer,0.993668,0.995776,0.991569,0.995776
3,aid_related,0.488885,0.58615,0.533326,0.58615
4,medical_help,0.889877,0.925627,0.856786,0.925627
5,medical_products,0.929908,0.952893,0.908005,0.952893
6,search_and_rescue,0.95948,0.972862,0.946461,0.972862
7,security,0.975108,0.983359,0.966995,0.983359
8,military,0.950928,0.967102,0.935286,0.967102
9,water,0.909184,0.938812,0.881368,0.938812


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
def average_f1_score(y_test, y_pred):

    """
    F1 scoring function for Grid Search;
    Calculates f1 scores for all labels and then calculate the average
    """

    f1_scores = []
    for i in range(y_pred.shape[1]):
        f1_scores.append(f1_score(np.array(y_test)[:,i], y_pred[:,i], average='weighted'))
    return np.mean(f1_scores)

In [19]:
f1_scorer = make_scorer(average_f1_score, greater_is_better=True)

In [20]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__fit_prior': [True, False]
}

cv = GridSearchCV(pipeline, parameters, cv=3, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [21]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  7.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                         

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
y_pred_cv = cv.predict(X_test)

In [24]:
results = evaluate_model(y_test, y_pred_cv)
results

Average f_score: 0.8998572797681685
Average accuracy: 0.9185502157852385
Average precision: 0.8878771032491966
Average recall: 0.9185502157852385


,Category,f_score,accuracy,precision,recall
0,related,0.679681,0.741039,0.67278,0.741039
1,request,0.795943,0.814516,0.785121,0.814516
2,offer,0.991487,0.991423,0.991551,0.991423
3,aid_related,0.541707,0.563108,0.540424,0.563108
4,medical_help,0.888334,0.918971,0.865887,0.918971
5,medical_products,0.927487,0.946749,0.910678,0.946749
6,search_and_rescue,0.956995,0.967486,0.947053,0.967486
7,security,0.972477,0.978111,0.966908,0.978111
8,military,0.948124,0.961086,0.935881,0.961086
9,water,0.906125,0.931388,0.884125,0.931388


In [25]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1f32fb00>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0,
                                          

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
clf = RandomForestClassifier(n_estimators=5)

In [28]:
pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('clf', MultiOutputClassifier(clf))
                    ])

In [29]:
pipeline_2.fit(X_train, y_train)
y_pred = pipeline_2.predict(X_test)

In [30]:
results = evaluate_model(y_test, y_pred)
results

Average f_score: 0.8988102807654543
Average accuracy: 0.9164143076585473
Average precision: 0.886512467991263
Average recall: 0.9164143076585473


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,f_score,accuracy,precision,recall
0,related,0.681682,0.72427,0.66683,0.72427
1,request,0.781251,0.813492,0.765778,0.813492
2,offer,0.993604,0.995648,0.991569,0.995648
3,aid_related,0.540937,0.553507,0.53742,0.553507
4,medical_help,0.883472,0.908986,0.861604,0.908986
5,medical_products,0.925508,0.942524,0.910098,0.942524
6,search_and_rescue,0.957972,0.969406,0.947423,0.969406
7,security,0.973441,0.980031,0.96694,0.980031
8,military,0.949197,0.963646,0.935176,0.963646
9,water,0.905561,0.929467,0.885045,0.929467


In [31]:
parameters_2 = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [5, 10, 25]
}

cv2 = GridSearchCV(pipeline_2, parameters_2, cv=3,scoring=f1_scorer, n_jobs=-1, verbose=1)

In [32]:
cv2.fit(X_train, y_train)
y_pred_cv2 = cv2.predict(X_test)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 27.7min finished


In [34]:
results = evaluate_model(y_test, y_pred_cv2)
results

Average f_score: 0.9004092614764765
Average accuracy: 0.9236047107014849
Average precision: 0.8887462542101981
Average recall: 0.9236047107014849


,Category,f_score,accuracy,precision,recall
0,related,0.679352,0.740527,0.671948,0.740527
1,request,0.782822,0.829493,0.775299,0.829493
2,offer,0.993668,0.995776,0.991569,0.995776
3,aid_related,0.551119,0.565924,0.548628,0.565924
4,medical_help,0.889468,0.923835,0.865128,0.923835
5,medical_products,0.929901,0.952381,0.915944,0.952381
6,search_and_rescue,0.959416,0.972734,0.946458,0.972734
7,security,0.974916,0.982975,0.966989,0.982975
8,military,0.950985,0.966718,0.941969,0.966718
9,water,0.907711,0.935868,0.881199,0.935868


### 9. Export your model as a pickle file

In [35]:
import joblib

In [36]:
joblib.dump(clf, '../models/disaster_response_prediction.pkl')

['../models/disaster_response_prediction.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.